# Database Normalization: Hands-on Laboratory

In this notebook, we will explore **why** we normalize databases. It's often hard to understand the value of normalization until you see things go wrong. 

We will move step-by-step through the Normal Forms:
1.  **First Normal Form (1NF)**: Atomicity and repeating groups.
2.  **Second Normal Form (2NF)**: Removing partial dependencies.
3.  **Third Normal Form (3NF)**: Removing transitive dependencies.

---

In [ ]:
import pandas as pd
import numpy as np

# Set display options to make tables look nice
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


## Part 1: First Normal Form (1NF) - The Foundation

Before we even talk about dependencies, we must have a valid table. 
**Rule 1NF:** All column values must be **atomic** (single values). No repeating groups or lists in a cell.

Let's look at a truly "Unnormalized" table.

In [ ]:
# Create an Unnormalized Dataset (UNF)
unf_data = {
    'Student': ['Alice', 'Bob', 'Charlie'],
    'Courses_Taken': ['Math 101, Physics 101', 'Math 101', 'History 101']
}

unf_df = pd.DataFrame(unf_data)
print("--- Unnormalized Form (Repeating Groups) ---")
display(unf_df)


### Fixing 1NF
This table violates 1NF because `Courses_Taken` contains multiple values. We cannot query "Who takes Physics 101?" easily without string manipulation.

**Solution:** We 'explode' the list so each course gets its own row.

In [ ]:
# Convert comma-separated string to list
df_1nf = unf_df.assign(Courses_Taken=unf_df['Courses_Taken'].str.split(', ')).explode('Courses_Taken')

print("--- First Normal Form (Atomic Values) ---")
display(df_1nf)

## Part 2: Second Normal Form (2NF) - Partial Dependencies

Now let's expand our 1NF table with more data. We have `Student_ID`, `Student_Name`, `Course`, `Instructor`, and `Instructor_Office`.

This table is in 1NF (all atomic), but fails 2NF.

In [ ]:
# Create the Expanded 1NF Dataset (but bad 2NF)
data_2nf_problem = {
    'Student_ID': [1, 1, 2, 3, 4],
    'Student_Name': ['Alice', 'Alice', 'Bob', 'Charlie', 'David'],
    'Course': ['Math 101', 'Physics 101', 'Math 101', 'History 101', 'Physics 101'],
    'Instructor': ['Dr. Jones', 'Dr. Smith', 'Dr. Jones', 'Dr. Brown', 'Dr. Smith'],
    'Instructor_Office': ['Rm 101', 'Rm 202', 'Rm 101', 'Rm 305', 'Rm 202']
}

poor_design_df = pd.DataFrame(data_2nf_problem)
print("--- 1NF Table (But violates 2NF) ---")
display(poor_design_df)

### The Partial Dependency Problem

The Primary Key is the combination **(Student_ID, Course)** because a student can take multiple courses.

**Violation:** `Student_Name` depends ONLY on `Student_ID`, which is just *part* of the key. It does not depend on the Course.

**Solution:** Move pure student data to its own table.

In [ ]:
# Create Students Table (Student_ID -> Name)
students_table = poor_design_df[['Student_ID', 'Student_Name']].drop_duplicates()

# Create Enrolments Table (Student_ID, Course, Instructor...)
enrolments_table = poor_design_df[['Student_ID', 'Course', 'Instructor', 'Instructor_Office']]

print("--- Students Table (2NF Compliant) ---")
display(students_table)

print("\n--- Enrolments Table (2NF Compliant, but fails 3NF) ---")
display(enrolments_table)

## Part 3: Third Normal Form (3NF) - Transitive Dependencies

Look at the `Enrolments Table` above. 
The Primary Key is `(Student_ID, Course)`.

**Violation:** `Instructor_Office` depends on `Instructor`. And `Instructor` depends on `Course`.
Only `Course` depends on the key (indirectly).

When a non-key attribute (`Office`) depends on another non-key attribute (`Instructor`), this is a **Transitive Dependency**.

**Scenario:** If Dr. Jones moves offices, we have an **Update Anomaly**. We'd have to update every row where he teaches.

In [ ]:
# Demonstration of Update Anomaly
print("--- Creating an Update Anomaly ---")
# We update distinct rows where Instructor is Dr. Jones, but miss one
enrolments_table.loc[(enrolments_table['Instructor'] == 'Dr. Jones') & (enrolments_table['Student_ID'] == 1), 'Instructor_Office'] = 'Rm 105'
display(enrolments_table)
print("Wait... Dr. Jones is in Room 105 AND Room 101? Inconsistent!")

### Fixing 3NF
We must separate the Instructor data into its own table.

In [ ]:
# 1. Create Instructors Table (Instructor -> Office)
instructors_table = pd.DataFrame({
    'Instructor': ['Dr. Jones', 'Dr. Smith', 'Dr. Brown'],
    'Office': ['Rm 101', 'Rm 202', 'Rm 305']
})

# 2. Create Courses Table (Course -> Instructor)
courses_table = pd.DataFrame({
    'Course_ID': ['Math 101', 'Physics 101', 'History 101'],
    'Instructor': ['Dr. Jones', 'Dr. Smith', 'Dr. Brown']
})

# 3. Create Final Enrolment Table (Student <-> Course)
final_enrolments = pd.DataFrame({
    'Student_ID': [1, 1, 2, 3, 4],
    'Course_ID': ['Math 101', 'Physics 101', 'Math 101', 'History 101', 'Physics 101']
})

print("--- Final 3NF Schema ---")
print("Instructors:"); display(instructors_table)
print("Courses:"); display(courses_table)
print("Enrolments:"); display(final_enrolments)
print("Students"); display(students_table)

### Proving the Fix

Now, if Dr. Jones moves to Room 105, we update ONE place.

In [ ]:
# Update Dr. Jones' office in the single source of truth
instructors_table.loc[instructors_table['Instructor'] == 'Dr. Jones', 'Office'] = 'Rm 105'

print("--- Instructors Table After Update ---")
display(instructors_table)

# Reconstruct the view to prove everything is updated everywhere
view = final_enrolments.merge(courses_table, on='Course_ID').merge(instructors_table, on='Instructor').merge(students_table, on='Student_ID')
print("\n--- Reconstructed View (Consistent!) ---")
display(view.sort_values('Student_ID'))

## Summary

*   **1NF**: No lists in cells (Atomic).
*   **2NF**: Student Name moved away from Course data (No Partial Dependency).
*   **3NF**: Instructor Office moved away from Course data (No Transitive Dependency).

We now have a normalized, anomaly-free database structure.